<a href="https://colab.research.google.com/github/vimesh630/Revenue_Forecasting/blob/main/Feature_Engineering_for_Revenue_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import os

# Define File Paths

In [ ]:
input_file_path = "/content/drive/MyDrive/VERGER/Revenue_Forecasting/preprocessed_forecast_data.csv"
output_dir = "/content/drive/MyDrive/VERGER/Revenue_Forecasting"
os.makedirs(output_dir, exist_ok=True)

# Load Dataset

In [ ]:
df = pd.read_csv(input_file_path)
df['date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2) + '-01')
df.sort_values(by='date', inplace=True)

# Feature Engineering Function

In [ ]:
def add_features(data):
    data = data.copy()
    group_cols = ['Account', 'Product', 'Type']
    data.set_index('date', inplace=True)

    for col in ['Forecast_Revenue', 'Forecast_Quantity']:
        for window in [1, 2, 3]:
            data[f'{col}_lag_{window}'] = data.groupby(group_cols)[col].shift(window)
        for window in [3, 6]:
            data[f'{col}_roll_mean_{window}'] = (
                data.groupby(group_cols)[col].shift(1).rolling(window=window).mean().reset_index(0, drop=True)
            )
            data[f'{col}_roll_std_{window}'] = (
                data.groupby(group_cols)[col].shift(1).rolling(window=window).std().reset_index(0, drop=True)
            )

    data.reset_index(inplace=True)
    data['month'] = data['date'].dt.month
    data['quarter'] = data['date'].dt.quarter
    data['year'] = data['date'].dt.year
    data = pd.get_dummies(data, columns=['Account', 'Product', 'Type'], drop_first=True)

    return data